In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from triplet_preparation import inputs_from_tuples, tuples_from_file_array
from model_architecture import triplet_network_model

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

tf.__version__, tf.keras.__version__

('2.1.0', '2.2.4-tf')

In [9]:
%%time
'''Test Dataset'''

files = [
    os.path.abspath('../data/samples/lichess_db_standard_rated_2013-01-tuples.h5')
]
# get triplets
triplets = tuples_from_file_array(files, "tuples", tuple_indices=[0, 1, 2])
# prepare for nn
train_triplets, test_triplets = inputs_from_tuples(triplets, test_split=True, test_size=0.05)
len(train_triplets),train_triplets[0].shape, len(test_triplets), test_triplets[0].shape

<KeysViewHDF5 ['tuples_0', 'tuples_1']>
CPU times: user 39.9 s, sys: 242 ms, total: 40.2 s
Wall time: 40 s


(3, (104144, 773), 3, (5482, 773))

In [5]:
'''Initialize and use triplet network'''
input_shape = (773,)
embedding_size = 10
model = triplet_network_model(input_shape, embedding_size)
optimizer = keras.optimizers.Adam(lr = 0.00006)
model.compile(loss=None,optimizer=optimizer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       [(None, 773)]        0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     [(None, 773)]        0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     [(None, 773)]        0                                            
__________________________________________________________________________________________________
embedding_layer (Dense)         (None, 10)           7740        anchor_input[0][0]               
                                                                 positive_input[0][0]         

In [8]:
history = model.fit(train_triplets,
                    batch_size=64,
                    epochs=3) #,
                    #validation_data=([anc_test, pos_test, neg_test]))

print('\nhistory dict:', history.history)


Train on 104144 samples
Epoch 1/3
104144/104144 [==============================] - 4s 41us/sample - loss: 0.0163
Epoch 2/3
104144/104144 [==============================] - 4s 42us/sample - loss: 0.0088
Epoch 3/3
104144/104144 [==============================] - 4s 42us/sample - loss: 0.0048

history dict: {'loss': [0.016253718489059602, 0.008812256722287073, 0.00478538013233496]}


In [32]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate([anc_test, pos_test, neg_test], batch_size=128)
print('test loss:', results)


# Evaluate on test data
18073/18073 [==============================] - 0s 16us/sample - loss: 9.7712
test loss: 9.77121658493525


### OLD

In [9]:
'''Create Checkpoints during training'''

checkpoint_path = "checkpoints/test_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
history = model.fit(train_images, 
                    train_labels,  
                    epochs=10,
                    validation_data=(test_images,test_labels),
                    callbacks=[cp_callback])  # Pass callback to training


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.2638 - accuracy: 0.6448
Epoch 00001: saving model to checkpoints/test_cp.ckpt
1000/1000 [==============================] - 1s 653us/sample - loss: 1.2335 - accuracy: 0.6530 - val_loss: 0.7204 - val_accuracy: 0.7850
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4300 - accuracy: 0.8792
Epoch 00002: saving model to checkpoints/test_cp.ckpt
1000/1000 [==============================] - 0s 296us/sample - loss: 0.4227 - accuracy: 0.8810 - val_loss: 0.5260 - val_accuracy: 0.8370
Epoch 3/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.3052 - accuracy: 0.9167
Epoch 00003: saving model to checkpoints/test_cp.ckpt
1000/1000 [==============================] - 0s 332us/sample - loss: 0.2916 - accuracy: 0.9230 - val_loss: 0.4594 - val_accuracy: 0.8510
Epoch 4/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.1980 - accu

In [10]:
'''Loads the weights from checkpoint path'''
model = create_model()
model.load_weights(checkpoint_path)

In [11]:
'''Save the complete model after training'''
model.save('model/test_model', save_format='tf') # or save_format='h5'

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model/test_model/assets


In [12]:
'''Load model from saved state'''
model = tf.keras.models.load_model('model/test_model')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [13]:
'''Use loaded model for inference'''
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))
print(model.predict(test_images).shape)

1000/1000 - 0s - loss: 0.3975 - accuracy: 0.8780
Restored model, accuracy: 87.80%
(1000, 10)
